In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, CuDNNLSTM, LSTM, Lambda
from keras import optimizers
from keras import callbacks
import numpy as np
from keras_tqdm import TQDMNotebookCallback
from sklearn.utils import shuffle
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [ ]:
# code used liberally from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

sonnets = []
with open("../data/shakespeare.txt") as f:
    line = f.readline()
    while line:
        # Flag start of sonnet, read in next 14 lines
        if any(char.isdigit() for char in line):
            curr_sonnet = ""
            for i in range(14):
                curr_sonnet += f.readline().strip().lower()
                curr_sonnet += "\n" if i != 13 else ""
            sonnets.append(curr_sonnet)
        line = f.readline()
        
# Vectorization prep
chars = sorted(list(set("".join(sonnets))))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

# Read subsequences from each sonnet, add to training list
# Don't read across sonnets?
length = 40
step = 1
tr_data = []
tar_char = []
for s in sonnets:
    for i in range(0, len(s) - length, step):
        tr_data.append(s[i:i+length])
        tar_char.append(s[i+length])
        
tr_data_full = []
tar_char_full = []
sonnets_full = "\n".join(sonnets)
for i in range(0, len(sonnets_full) - length, step):
    tr_data_full.append(sonnets_full[i:i+length])
    tar_char_full.append(sonnets_full[i+length])
    
# Vectorize training data
X = np.zeros((len(tr_data), length, len(chars)), dtype=np.bool)
Y = np.zeros((len(tr_data), len(chars)), dtype=np.bool)

X_full = np.zeros((len(tr_data_full), length, len(chars)), dtype=np.bool)
Y_full = np.zeros((len(tr_data_full), len(chars)), dtype=np.bool)

for i, seq in enumerate(tr_data):
    for j, char in enumerate(seq):
        X[i, j, char_index[char]] = 1
    Y[i, char_index[tar_char[i]]] = 1
    
for i, seq in enumerate(tr_data_full):
    for j, char in enumerate(seq):
        X_full[i, j, char_index[char]] = 1
    Y_full[i, char_index[tar_char_full[i]]] = 1
    
X_full_shuff, Y_full_shuff = shuffle(X_full, Y_full)